In [ ]:
!pip install gpytorch
!pip install tqdm
!pip install -U scikit-learn
!pip install properscoring

In [ ]:
import math
import torch
import gpytorch
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import tqdm
import sklearn
import copy
import properscoring as ps
%matplotlib inline
%load_ext autoreload
%autoreload 2
from torch.distributions.transformed_distribution import TransformedDistribution
from torch.distributions.transforms import SigmoidTransform, PowerTransform
from torch.distributions.normal import Normal

In [ ]:
def fill_ndarray(t1):
    for i in range(t1.shape[1]):  
        temp_col = t1[:, i]  
        nan_num = np.count_nonzero(temp_col != temp_col)
        if nan_num != 0:  
            temp_not_nan_col = temp_col[temp_col == temp_col]  
            temp_col[np.isnan(temp_col)] = temp_not_nan_col.mean()  
    return t1

In [ ]:
def get_data(df):
  data = df['TARGETVAR'].values.reshape((-1,1))
  return data

def get_nwp(d):
  cls = d.columns
  data = []
  for i in range(4):
    data.append(d[cls[i+3]].values.reshape((-1,1)))
  data = np.hstack(data)
  return data

data = pd.read_csv('Task15_W_Zone1.csv',delimiter=',')

power = get_data(data); x = get_nwp(data)
y = fill_ndarray(power).flatten()
y_true = copy.deepcopy(y)

v = 1
for i in range(len(y)):
    if y[i] == 0:
        y[i] = 1e-3
        y_true[i] = 1e-3
        y[i] = np.log(np.power(y[i],v)/(1-np.power(y[i],v)))
    elif y[i] == 1:
        y[i] = 1-1e-3
        y_true[i] = 1-1e-3
        y[i] = np.log(np.power(y[i],v)/(1-np.power(y[i],v)))
    else:
        y[i] = np.log(np.power(y[i],v)/(1-np.power(y[i],v)))
y = torch.tensor(y).float()
x = torch.tensor(x).float()

samples = math.floor(x.shape[0]*0.8)

train_y = y[:samples]
train_x = x[:samples,:]


y_true = y_true[samples:]
test_y = y[samples:]
test_x = x[samples:,:]

if torch.cuda.is_available():
    train_x, train_y, test_x, test_y = train_x.cuda(), train_y.cuda(), test_x.cuda(), test_y.cuda()

In [ ]:
from torch.utils.data import TensorDataset, DataLoader
train_dataset = TensorDataset(train_x, train_y)
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)

test_dataset = TensorDataset(test_x, test_y)
test_loader = DataLoader(test_dataset, batch_size=1024, shuffle=False)

In [ ]:
from torch import nn
class MLPMean(gpytorch.means.Mean):
    def __init__(self, input_size):
        super(MLPMean, self).__init__()
        self.mlp = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),  
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(), 
            nn.Linear(512, 1))

        count = 0
        for n, p in self.mlp.named_parameters():
            self.register_parameter(name = 'mlp' + str(count), parameter = p)
            count += 1
    
    def forward(self, x):
        m = self.mlp(x)
        return m.squeeze()

In [ ]:
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy

class GPModel(ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(GPModel, self).__init__(variational_strategy)
        self.scale_to_bounds = gpytorch.utils.grid.ScaleToBounds(-1., 1.)
        
        self.mean_module = MLPMean(input_size=4)
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=4))
        

    def forward(self, x):
        x = self.scale_to_bounds(x)
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

inducing_points = train_x[torch.randperm(train_x.size(0))[:1000]]
model = GPModel(inducing_points=inducing_points)
likelihood = gpytorch.likelihoods.GaussianLikelihood()

if torch.cuda.is_available():
    model = model.cuda()
    likelihood = likelihood.cuda()

In [ ]:
num_epochs = 200
losses = []

model.train()
likelihood.train()

optimizer = torch.optim.Adam([
    {'params': model.parameters()},
    {'params': likelihood.parameters()},
], lr=0.001)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer=optimizer, milestones=[50,100], gamma=0.5)

mll = gpytorch.mlls.PredictiveLogLikelihood(likelihood, model, num_data=train_y.size(0))


epochs_iter = tqdm.notebook.tqdm(range(num_epochs), desc="Epoch")
for i in epochs_iter:
    minibatch_iter = tqdm.notebook.tqdm(train_loader, desc="Minibatch", leave=False)
    for x_batch, y_batch in minibatch_iter:
        optimizer.zero_grad()
        output = likelihood(model(x_batch))
        loss = -mll(output, y_batch)
        minibatch_iter.set_postfix(loss=loss.item())
        loss.backward()
        optimizer.step()
    scheduler.step()
    losses.append(loss.item())


In [ ]:
model.eval()
likelihood.eval()

mean = torch.tensor([0.]); std = torch.tensor([0.]); f_std = torch.tensor([0.])
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        preds = likelihood(model(x_batch))
        mean = torch.cat([mean, preds.mean.cpu()])
        std = torch.cat([std, preds.stddev.cpu()])
        f_std = torch.cat([f_std, model(x_batch).stddev.cpu()])
mean = mean[1:]; std = std[1:]; f_std = f_std[1:]

In [ ]:
mean = mean.cpu()
std = std.cpu()
f_std = f_std.cpu()

train_x = train_x.cpu()
train_y = train_y.cpu()
test_x = test_x.cpu()
test_y = test_y.cpu()

In [ ]:
length = len(y_true)
score = 0 
for i in range(length):
  m = torch.distributions.normal.Normal(mean[i],std[i])
  transforms = [SigmoidTransform(),PowerTransform(1)]
  logit_normal = TransformedDistribution(m, transforms)
  sample = logit_normal.sample([1000]).numpy()
  score += ps.crps_ensemble(y_true[i] , sample)

score = score/length
print(score)

0.10304097427849318
